# Detectron2

In [1]:
#!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

#exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
!pip install opencv-python
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html


In [2]:
#uncoment when working on Google Colab
from google.colab import drive
drive.mount("/content/gdrive")
print(os.getcwd())

Mounted at /content/gdrive
/content


I used https://plainsight.ai to perform polygon annotations and exported the data utilizing their COCO format feature. However, the images segmentations are exported in the following format "segmentation" : [[x_1, y_1], [x_2, y_2], ...] which is not the forma expected from detectron2. Thefore, we first need to flatten the segmentation array of arrays prior to registering with detectron. The cell below takes care of that

In [3]:
import json
import numpy as np


def convert_to_COCO(json_path):
  #variable where we will save our new json object
    new_json = None
    
    with open(json_path, 'r') as file:
        json_object = json.load(file)
        annotations = json_object['annotations']
    #function to flatten the array of arrays
    for i in range(0, len(annotations)):
        annotations[i]["segmentation"]  =  [np.array(annotations[i]["segmentation"]).flatten().tolist()]

    #save to json object 
    json_object['annotations'] = annotations

    new_json = json_object
    return new_json


#function to save a json file to selected path, note we should append the name of the file
def save_JSON(data, path):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)


#Google colab path
#dataset_path = "/content/gdrive/MyDrive/Spring 2022 Courses/Neural Networks/Amazon-Robotics-snapshot-001-project-coco-1646593032"
dataset_path = '/content/gdrive/MyDrive/Amazon Project/clutterized/'
#scratch365 path
#dataset_path = "./Amazon-Robotics-snapshot-001-project-coco-1646593032"
old_annotations_train = dataset_path + "/train.json"
image_path = dataset_path 


new_json_train = convert_to_COCO(old_annotations_train)
save_JSON(new_json_train, dataset_path+"new_train.json")
annotations_train = dataset_path + "new_train.json"



If we want to use a custom dataset while also reusing detectron2’s data loaders, you will need to:

1.   Register your dataset (i.e., tell detectron2 how to obtain your dataset).
2.   Optionally, register metadata for your dataset.

It contains a mapping from strings (which are names that identify a dataset, e.g. “coco_2014_train”) to a function which parses the dataset and returns the samples in the format of list[dict]. 

For more details: https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html

In [4]:
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode

#telling detectron what to call my training dataset, path to json file, and path to images
register_coco_instances("my_dataset_train", {}, annotations_train, image_path)

#visualize training data
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset = DatasetCatalog.get("my_dataset_train") #Call the registered function and return its results (return list[dict] – dataset annotations.)

[04/24 15:57:57 d2.data.datasets.coco]: Loaded 44 images in COCO format from /content/gdrive/MyDrive/Amazon Project/clutterized/new_train.json


Here we visualize random sample from our dataset, including the bbox, polygon segmentation and it's label

In [5]:
import random
from detectron2.utils.visualizer import Visualizer
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

rows, cols = 4, 3
j = 0
fig = plt.figure(figsize=(45,45))

for d in random.sample(train_dataset, 11): #returns 11 random samples from the training dataset
    img = cv2.imread(d["file_name"]) #respective file name
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.2)
    out = visualizer.draw_dataset_dict(d)
    fig.add_subplot(rows, cols, j+1)
    j = j+ 1
    plt.imshow(out.get_image()[:, :, ::-1])


plt.show()


Output hidden; open in https://colab.research.google.com to view.

# Fine Tunning and Training

In [6]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 600  # 600 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 200  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = '/content/gdrive/MyDrive/Amazon Project/clutterized/output/'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
#trainer = CustomTrainer(cfg)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[04/24 15:58:29 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:04, 39.0MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due 

[04/24 15:58:39 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/24 15:59:01 d2.utils.events]:  eta: 0:10:31  iter: 19  total_loss: 4.331  loss_cls: 2.549  loss_box_reg: 0.9227  loss_mask: 0.6927  loss_rpn_cls: 0.1782  loss_rpn_loc: 0.05393  time: 1.0924  data_time: 0.5459  lr: 8.1588e-06  max_mem: 2549M
[04/24 15:59:20 d2.utils.events]:  eta: 0:09:15  iter: 39  total_loss: 4.085  loss_cls: 2.333  loss_box_reg: 0.9114  loss_mask: 0.6892  loss_rpn_cls: 0.121  loss_rpn_loc: 0.04516  time: 1.0082  data_time: 0.3655  lr: 1.6484e-05  max_mem: 2549M
[04/24 15:59:38 d2.utils.events]:  eta: 0:08:42  iter: 59  total_loss: 3.746  loss_cls: 1.903  loss_box_reg: 0.9199  loss_mask: 0.6836  loss_rpn_cls: 0.1045  loss_rpn_loc: 0.04829  time: 0.9688  data_time: 0.3371  lr: 2.4809e-05  max_mem: 2549M
[04/24 15:59:56 d2.utils.events]:  eta: 0:08:10  iter: 79  total_loss: 3.102  loss_cls: 1.404  loss_box_reg: 0.9273  loss_mask: 0.6739  loss_rpn_cls: 0.0694  loss_rpn_loc: 0.04122  time: 0.9475  data_time: 0.3284  lr: 3.3134e-05  max_mem: 2549M
[04/24 16:00:14 d2.ut

# Training Curves

In [7]:
# Look at training curves in tensorboard:
#%load_ext tensorboard
#%tensorboard --logdir output